<a href="https://colab.research.google.com/github/usma11dia0/web_scraping_on_colab/blob/main/yahoo_scraping_outdoor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.yEvIbQ9clW/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.NgoPMlDqfS/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.9kIGp7hif8/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [ ]:
#環境変数設定
TARGET = 'ダイエット・ヘルスケア'
TARGET_URL = 'https://shopping.yahoo.co.jp/category/2052/list?sc_i=shp_pc_catelist_nrwcgt'

NUM_TO_FETCH = 200
STORES_PER_PAGE = 30
DRIVER_WAIT_TIME = 2
SCROLL_PAUSE_TIME = 3
MAX_SCROLL_ATTEMPTS = 5

# 各種ファイル保存先
ACCU_LIST_PATH = '/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_yahoo/accumulated_list_yahoo_outdoor.csv'
SUBMIT_FILE_PATH ='/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_yahoo/result_list_yahoo_diet_nurse.xlsx'

#取得データ格納先
result_dict = { 
    '会社名(ストア情報)': [],
    '住所(ストア情報)': [],
    'ストア名(ストア情報)':[],
    '運営責任者(問い合わせ情報)':[],
    '住所(問い合わせ情報)':[],
    '電話番号(問い合わせ情報)':[],
    'メールアドレス(問い合わせ情報)':[],
    'URL':[],
}

In [ ]:
#標準ライブラリ
import os
import math
import json
import re
import time

#サードパーティライブラリ
import pandas as pd
from google.colab import drive
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from logging import (
    getLogger, 
    StreamHandler, 
    DEBUG, 
    INFO, 
    Formatter, 
    config,
)
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#関数定義
def add_store_url(elements, store_urls, store_names, count_skip):
    for i, element in enumerate(elements, start = 0):
        if i > 29:
          break
        store_name = element.text
        store_url = element.get_attribute('href')
        if store_name not in accu_set and store_name not in store_names:
            store_urls.append(store_url)
            store_names.append(store_name)
        else:
            count_skip += 1
    return store_urls, store_names, count_skip

#スクロールしてページを更新
def scroll_page(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    attempts = 0
    while attempts < MAX_SCROLL_ATTEMPTS:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        attempts += 1

def is_japanese_phone_number(phone_number):
    # 固定電話のパターン
    landline_pattern = r'^0\d{1,4}-\d{1,4}-\d{3,4}$'
    # 携帯電話のパターン
    mobile_pattern = r'^0[789]0-\d{4}-\d{4}$'

    return bool(re.match(landline_pattern, phone_number) or re.match(mobile_pattern, phone_number))

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--ignore-certificate-errors')
options.add_argument("--disable-extensions")
options.add_argument("--disable-popup-blocking")
options.add_argument(
       "user-agent=Mozilla/5.0 (X11; Linux x86_64; rv:93.0) Gecko/20100101 Firefox/93.0"
    )
options.add_experimental_option("prefs", {
    "profile.managed_default_content_settings.images": 2,  # 画像の無効化
    "profile.managed_default_content_settings.plugins": 2,  # プラグインの無効化
})

driver = webdriver.Chrome('chromedriver', options=options)
driver.implicitly_wait(DRIVER_WAIT_TIME)

In [ ]:
#ロガーの設定
with open('/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_yahoo/logging_config.json', 'r') as f:
    logger_config = json.load(f)
config.dictConfig(logger_config)
logger = getLogger('main')

try:
      driver.get(TARGET_URL)
      logger.debug(f'{TARGET}：トップページへ移動しました')
except Exception as e:
      logger.error(f'{TARGET}：トップページへ移動出来ませんでした: [e]')
      raise

2023-05-18 10:40:55,152 [DEBUG] main: ダイエット・ヘルスケア：トップページへ移動しました


In [ ]:
#店舗リストの全ページ数を導出
p_element = driver.find_element(By.CSS_SELECTOR,".SearchResultsDisplayOptions_SearchResultsDisplayOptions__count__WBsPf")
num_all_stores = int(p_element.text.replace(',','')[:-1])
num_all_pages = math.ceil(num_all_stores / STORES_PER_PAGE)


#積み上げリストから既存取得の店舗URLを抽出
try:
  df_accu = pd.read_csv(ACCU_LIST_PATH, usecols=[2])
  accu_list = df_accu.iloc[:, 0].values.tolist()
  accu_set = set(accu_list)
except FileNotFoundError:
  accu_set = set()

# 目標件数を満たすまでstore_urlを取得
try:
  store_urls = []
  store_names = []
  count = 0
  count_skip = 0
  while len(store_urls) < NUM_TO_FETCH:
    start = 30 * count if count != 0 else 0
    scroll_page(driver)
    div_elements = driver.find_elements(By.CSS_SELECTOR,".SearchResult_SearchResult__storeRow__tXYSC a")
    div_elements = div_elements[start:]
    #ページ末尾までクロールが終了した場合、次のページへ遷移
    if len(div_elements) == 0:
      try:
        page_elements = driver.find_elements(By.CSS_SELECTOR, ".Pager_Pager__lbzCH")
        page_element = page_elements[-1]
        a_element = page_element.find_element(By.CSS_SELECTOR, "div:nth-child(3) a")
        page_url = a_element.get_attribute('href')
        driver.get(page_url)
        scroll_page(driver)
        count = 0
        start = 0
        div_elements = driver.find_elements(By.CSS_SELECTOR,".SearchResult_SearchResult__storeRow__tXYSC a")
        div_elements = div_elements[start:]
        logger.debug(f'{TARGET}：次の店舗一覧ページへ遷移しました)')
        print(page_url)
      except:
        logger.debug(f'{TARGET}：次の店舗一覧ページへ遷移出来ませんでした)')
    store_urls, store_names, count_skip = add_store_url(div_elements, store_urls, store_names, count_skip)
    logger.debug(f'{TARGET}：店舗リスト取得 (取得数:{len(store_urls)}, 重複数：{count_skip})')
    count += 1
except KeyboardInterrupt:
  logger.info(f'{TARGET}：店舗URL取得を中断しました:{len(store_urls)}個') 
  print(store_urls)
  print(store_names)
  driver.quit()
  raise
except  Exception as e:
  logger.error(f'{TARGET}：店舗リストへ移動出来ませんでした: [{e}]')
  driver.quit()
  raise
logger.info(f'{TARGET}：店舗URL取得完了:{len(store_urls)}個')  
print(store_urls)
print(store_names)

2023-05-18 10:41:13,966 [DEBUG] main: ダイエット・ヘルスケア：店舗リスト取得 (取得数:0, 重複数：30)
2023-05-18 10:41:34,609 [DEBUG] main: ダイエット・ヘルスケア：店舗リスト取得 (取得数:0, 重複数：60)
2023-05-18 10:42:04,026 [DEBUG] main: ダイエット・ヘルスケア：店舗リスト取得 (取得数:0, 重複数：90)
2023-05-18 10:42:33,817 [DEBUG] main: ダイエット・ヘルスケア：店舗リスト取得 (取得数:1, 重複数：119)
2023-05-18 10:42:47,107 [DEBUG] main: ダイエット・ヘルスケア：店舗リスト取得 (取得数:1, 重複数：149)
2023-05-18 10:42:51,075 [DEBUG] main: ダイエット・ヘルスケア：店舗リスト取得 (取得数:1, 重複数：179)
2023-05-18 10:42:55,071 [DEBUG] main: ダイエット・ヘルスケア：店舗リスト取得 (取得数:1, 重複数：209)
2023-05-18 10:42:58,939 [DEBUG] main: ダイエット・ヘルスケア：店舗リスト取得 (取得数:1, 重複数：239)
2023-05-18 10:43:02,725 [DEBUG] main: ダイエット・ヘルスケア：店舗リスト取得 (取得数:1, 重複数：269)
2023-05-18 10:43:06,540 [DEBUG] main: ダイエット・ヘルスケア：店舗リスト取得 (取得数:1, 重複数：299)
2023-05-18 10:43:10,615 [DEBUG] main: ダイエット・ヘルスケア：店舗リスト取得 (取得数:1, 重複数：329)
2023-05-18 10:43:14,773 [DEBUG] main: ダイエット・ヘルスケア：店舗リスト取得 (取得数:1, 重複数：359)
2023-05-18 10:43:18,801 [DEBUG] main: ダイエット・ヘルスケア：店舗リスト取得 (取得数:1, 重複数：389)
2023-05-18 10:43:23,367 [DEB

MaxRetryError: ignored

In [ ]:
page_num = 0
#店舗個別ページにて情報取得
for store_url in store_urls:   
    name_store = "-"
    address_store = "-"
    store_store = "-"
    name_contact = "-"
    address_contact = "-"
    tel_contact = "-"
    mail_contact = "-"
    try:
        driver.get(store_url)
        page_num += 1
        store_store = store_names[page_num - 1]
        logger.debug(f'{TARGET}：店舗ページ{page_num}へ移動しました')

        #店舗情報ページURLを取得
        info_urls = driver.find_elements(By.CSS_SELECTOR,".elInfoItemLink")
        if not info_urls:
          info_urls = driver.find_elements(By.CSS_SELECTOR,".styles_infoItemLink__m2ij_")
        info_url = info_urls[0].get_attribute('href')
        print(info_url)
        driver.get(info_url)
        logger.debug(f'{TARGET}：店舗情報ページ{page_num}へ移動しました')
        name_store = driver.find_element(By.CSS_SELECTOR, ".elSectionContent .elRow:nth-child(1) .elRowContent .elRowContentText").text
        address_store = driver.find_element(By.CSS_SELECTOR, ".elSectionContent .elRow:nth-child(3) .elRowContent .elRowContentText").text
        div_elements =  driver.find_elements(By.CSS_SELECTOR, ".mdInformationTable")
        div_element = div_elements[1]
        name_contact = div_element.find_element(By.CSS_SELECTOR, ".elRows .elRow:nth-child(1) .elRowContentText").text
        address_contact = div_element.find_element(By.CSS_SELECTOR, ".elRows .elRow:nth-child(3) .elRowContentText").text
        tel_contact = div_element.find_element(By.CSS_SELECTOR, ".elRows .elRow:nth-child(4) .elRowContentText").text
        mail_contact = div_element.find_element(By.CSS_SELECTOR, ".elRows .elRow:nth-child(5) .elRowContentText").text
        if is_japanese_phone_number(mail_contact):
          mail_contact = div_element.find_element(By.CSS_SELECTOR, ".elRows .elRow:nth-child(6) .elRowContentText").text
    except Exception as e:
        logger.error(f'{TARGET}：会社概要ページがありませんでした。: [{store_store, store_url}]:[{e}]')
    finally:
      result_dict['会社名(ストア情報)'].append(name_store)
      result_dict['住所(ストア情報)'].append(address_store)
      result_dict['ストア名(ストア情報)'].append(store_store)
      result_dict['運営責任者(問い合わせ情報)'].append(name_contact)
      result_dict['住所(問い合わせ情報)'].append(address_contact)
      result_dict['電話番号(問い合わせ情報)'].append(tel_contact)
      result_dict['メールアドレス(問い合わせ情報)'].append(mail_contact)
      result_dict['URL'].append(store_url)

df_result = pd.DataFrame(result_dict)
df_result

2023-05-18 10:49:28,873 [DEBUG] main: ダイエット・ヘルスケア：店舗ページ1へ移動しました
https://store.shopping.yahoo.co.jp/paramountbed/info.html
2023-05-18 10:49:33,351 [DEBUG] main: ダイエット・ヘルスケア：店舗情報ページ1へ移動しました
2023-05-18 10:49:36,598 [DEBUG] main: ダイエット・ヘルスケア：店舗ページ2へ移動しました
https://store.shopping.yahoo.co.jp/shoesbase2nd/info.html
2023-05-18 10:49:41,145 [DEBUG] main: ダイエット・ヘルスケア：店舗情報ページ2へ移動しました
2023-05-18 10:49:44,122 [DEBUG] main: ダイエット・ヘルスケア：店舗ページ3へ移動しました
https://store.shopping.yahoo.co.jp/dogparadise/info.html
2023-05-18 10:49:48,348 [DEBUG] main: ダイエット・ヘルスケア：店舗情報ページ3へ移動しました
2023-05-18 10:49:51,450 [DEBUG] main: ダイエット・ヘルスケア：店舗ページ4へ移動しました
https://store.shopping.yahoo.co.jp/koyo-omutsu/info.html
2023-05-18 10:49:55,505 [DEBUG] main: ダイエット・ヘルスケア：店舗情報ページ4へ移動しました
2023-05-18 10:49:59,108 [DEBUG] main: ダイエット・ヘルスケア：店舗ページ5へ移動しました
2023-05-18 10:50:03,147 [ERROR] main: ダイエット・ヘルスケア：会社概要ページがありませんでした。: [('ココチのくらし雑貨店', 'https://store.shopping.yahoo.co.jp/e-zakkaya/?sc_i=shp_pc_catelist_itemlist_shsrg_strnm')]:[list ind

,会社名(ストア情報),住所(ストア情報),ストア名(ストア情報),運営責任者(問い合わせ情報),住所(問い合わせ情報),電話番号(問い合わせ情報),メールアドレス(問い合わせ情報),URL
0,パラマウントベッド株式会社,東京都江東区東砂2-14-5,パラマウントベッドストア,健康事業推進部 一木哲也,東京都中央区京橋１丁目６－１三井住友海上テプコビル１階,03-5250-1515,pbstore@paramount.co.jp,https://ck.storematch.jp/rd?v=4.2&c=4Q&campaig...
1,株式会社ジョイベ,福井県坂井市丸岡町東陽2-31シューズベース,シューズベースYahoo!店,西村 長浩,福井県坂井市丸岡町小黒64-1佐川急便(株) 丸岡営業所 2F,0776-89-1176,shoesbase@joybe.co.jp,https://store.shopping.yahoo.co.jp/shoesbase2n...
2,株式会社今人,埼玉県越谷市蒲生茜町28-21,ドッグパラダイスぷらすニャン,通販部 嶋田 淳,埼玉県越谷市蒲生茜町19-5ソライエアイル203,048-999-6829,shop@dogparadise.jp,https://store.shopping.yahoo.co.jp/dogparadise...
3,株式会社光洋,神奈川県横浜市金沢区福浦1-5-1,紙おむつの光洋,在宅サービス課 中島 美紀,神奈川県横浜市金沢区福浦 1-5-1,0120-21-3858,rakuten@koyo.jp,https://store.shopping.yahoo.co.jp/koyo-omutsu...
4,-,-,ココチのくらし雑貨店,-,-,-,-,https://store.shopping.yahoo.co.jp/e-zakkaya/?...
...,...,...,...,...,...,...,...,...
128,株式会社エーアンドエー,神奈川県横浜市中区豆口台144-3,カドクラ車椅子 公式ショップ,運営管理 二宮英玲奈/ 桜木陣宗,神奈川県横浜市中区扇町 1-1-16 DR204,045-681-5148,shopping@kadokura.org,https://store.shopping.yahoo.co.jp/xenashoppin...
129,田中伊吹,東京都練馬区東大泉7-50-40-5-202号大泉積徳マンション,キュレキュレ,田中 伊吹,東京都練馬区東大泉7-50-40-5-202号大泉積徳マンション,090-2935-8110,jiilz51109@yahoo.co.jp,https://store.shopping.yahoo.co.jp/cure-store/...
130,株式会社iHoldings,静岡県静岡市葵区北番町113-1,PrimeSellerJapan,深 沢 雄 次,静岡県静岡市葵区北番町113-1,054-204-5000,mail@iholdings.jp,https://store.shopping.yahoo.co.jp/primeseller...
131,有限会社ハートプランニング,三重県四日市市西坂部町3996-14,ハートプランニング,黒田 隆典,三重県三重郡菰野町小島3323,059-328-4112,heartplanning@mocha.ocn.ne.jp,https://store.shopping.yahoo.co.jp/hp100/?sc_i...


In [ ]:
# 提出ファイル用出力
if not os.path.isfile(SUBMIT_FILE_PATH):
    df_result.to_excel(SUBMIT_FILE_PATH, header=True, index=False)
else:
  with pd.ExcelWriter(SUBMIT_FILE_PATH, mode='a', if_sheet_exists='new') as writer:
    df_result.to_excel(writer)

# 積み上げリスト保存用出力
df_result.to_csv(ACCU_LIST_PATH, mode='a', header=True, index=False)

In [ ]:
# 積み上げリスト確認
df_accu = pd.read_csv(ACCU_LIST_PATH)
df_accu

,会社名(ストア情報),住所(ストア情報),ストア名(ストア情報),運営責任者(問い合わせ情報),住所(問い合わせ情報),電話番号(問い合わせ情報),メールアドレス(問い合わせ情報),URL
0,-,-,viva ヤフーショッピング店,-,-,-,-,https://store.shopping.yahoo.co.jp/vivaenterpl...
1,株式会社アイボリーライフ,兵庫県西宮市上甲子園四丁目8番9号,アイボリー製造所2,金子 達也,兵庫県西宮市上甲子園5丁目1-1,050-3592-2941,info@withyou-club.com,https://store.shopping.yahoo.co.jp/ivory-mp2/?...
2,-,-,Swa Suwa,-,-,-,-,https://store.shopping.yahoo.co.jp/swasuwa/?sc...
3,＆MAGIC,東京都狛江市駒井町3-7-15-101,三代目つるや洋品店,川口 樹,東京都狛江市駒井町3-7-15-101,050-3196-2220,tsuruya.the3rd@gmail.com,https://store.shopping.yahoo.co.jp/supermarket...
4,株式会社Pleasant.japan,東京都狛江市岩戸北1丁目10-6番地甲武ビル201,pleasant.japan,根本 洋一,東京都狛江市岩戸北1-10-6 甲武ビル201,03-5761-4839,info@pleasant-japan.com,https://store.shopping.yahoo.co.jp/pleasant-ja...
...,...,...,...,...,...,...,...,...
3260,株式会社エーアンドエー,神奈川県横浜市中区豆口台144-3,カドクラ車椅子 公式ショップ,運営管理 二宮英玲奈/ 桜木陣宗,神奈川県横浜市中区扇町 1-1-16 DR204,045-681-5148,shopping@kadokura.org,https://store.shopping.yahoo.co.jp/xenashoppin...
3261,田中伊吹,東京都練馬区東大泉7-50-40-5-202号大泉積徳マンション,キュレキュレ,田中 伊吹,東京都練馬区東大泉7-50-40-5-202号大泉積徳マンション,090-2935-8110,jiilz51109@yahoo.co.jp,https://store.shopping.yahoo.co.jp/cure-store/...
3262,株式会社iHoldings,静岡県静岡市葵区北番町113-1,PrimeSellerJapan,深 沢 雄 次,静岡県静岡市葵区北番町113-1,054-204-5000,mail@iholdings.jp,https://store.shopping.yahoo.co.jp/primeseller...
3263,有限会社ハートプランニング,三重県四日市市西坂部町3996-14,ハートプランニング,黒田 隆典,三重県三重郡菰野町小島3323,059-328-4112,heartplanning@mocha.ocn.ne.jp,https://store.shopping.yahoo.co.jp/hp100/?sc_i...
